## LLM FINE TUNING USING LORA AND QLORA

### 1.Install Required Packages AND Configuration setup

In [ ]:
# Install Required Packages
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

# Import Libraries
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
)
from peft import LoraConfig
from trl import SFTTrainer

# Configuration
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "Llama-2-7b-chat-finetune"


# Load Dataset with Train and Validation Splits
dataset = load_dataset(dataset_name, split="train")
dataset = dataset.train_test_split(test_size=0.1)  # 10% for evaluation
train_dataset = dataset['train']
eval_dataset = dataset['test']


### 2.LoRA and Quantization Parameters AND Tokenizer

In [ ]:
# LoRA and Quantization Parameters
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Load Model and Tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


### 3.TRAINING

In [ ]:
# Training Arguments
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,  # Batch size for evaluation
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    logging_steps=25,
    save_steps=0,
    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    evaluation_strategy="steps",  # Evaluate at intervals
    eval_steps=25,  # Evaluate every 25 steps
    do_eval=True  # Enable evaluation
)

# Fine-Tuning Setup
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Add evaluation dataset
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments
)

# Train and Save the Model
trainer.train()
trainer.model.save_pretrained(new_model)

### 4. Visualize Training Logs with TensorBoard

In [ ]:
# Import TensorBoard for visualization in Colab
%load_ext tensorboard

# Start TensorBoard (this will automatically open the UI in Colab)
%tensorboard --logdir ./results